# Définition du pricer selon le modèle de Black & Scholes 

In [105]:
import numpy as np
from scipy.stats import norm
import pandas as pd 
from datetime import datetime
from scipy.optimize import minimize

# Charger le fichier CSV en utilisant le chemin relatif
data = pd.read_csv("LigneOptionFuture.csv", sep = ",")
print(data.head())  # Affiche les premières lignes pour vérifier que le fichier a été chargé correctement


# Fonction pour calculer le taux de dividende
def div_yield(div_pv, S, ttm):
    return div_pv / (S * ttm)

# Fonction de pricing d'option Black-Scholes
def Pricer(S, K, r_d, ttm, div_pv, vol):
    div_yield_value = div_yield(div_pv, S, ttm)
    r_c = np.log(1+r_d)
    d1 = (np.log(S / K) + (r_c - div_yield_value + 0.5 * vol ** 2) * ttm) / (vol * np.sqrt(ttm))
    d2 = d1 - vol * np.sqrt(ttm)
    P_Call = S * np.exp(-div_yield_value * ttm) * norm.cdf(d1) - np.exp(-r_c * ttm) * K * norm.cdf(d2)
    P_Put = -S * np.exp(-div_yield_value * ttm) * norm.cdf(-d1) + np.exp(-r_c * ttm) * K * norm.cdf(-d2)
    DeltaCall = np.exp(-div_yield_value * ttm) * norm.cdf(d1)
    DeltaPut = -np.exp(-div_yield_value * ttm) * norm.cdf(-d1)
    return P_Call, P_Put, DeltaCall, DeltaPut

                             Code  Instrument_id                        Name  \
0  OPT_EURO-BUND_11/18_CALL_158.5          27085  EURO-BUND 11/18 CALL 158.5   

   Quantity  Close  Mo  underlying_id UnderlyingType         Underlying_Name  \
0        50   0.51   1          26936            FUC  EURO-BUND FUTURE Dec18   

   Underlying_Close  Strike Expiration_date  Days_To_Expiration TypeOption  \
0            158.04   158.5      2018-10-26              0.0603       CALL   

     Type     MV    Weight  
0  Option  25500  0.000677  


## Vérification du pricer

In [106]:
# Fonction pour calculer la différence entre le prix réel et le prix call calculé
def objective_function(vol):
    P_Call, _, _, _ = Pricer(Underlying_close, Strike, r_d, ttm, div_pv, vol)
    return (P_Call - Close) ** 2  # On minimise le carré de l'erreur

# Paramètres donnés
Close = data['Close'][0]
Underlying_close = data['Underlying_Close'][0]
Strike = data['Strike'][0]
ttm = data['Days_To_Expiration'][0]  # Conversion en années
r_d = 0
div_pv = 0
vol_initial = 0.5
Type = data['TypeOption'][0]

# Calculer les prix et les deltas avec la volatilité initiale
P_Call, P_Put, DeltaCall, DeltaPut = Pricer(Underlying_close, Strike, r_d, ttm, div_pv, vol_initial)

# Affichage des résultats formatés
print(f"{'Paramètre':<15}{'Valeur'}")
print("-" * 25)
print(f"{'Prix Call':<15}{P_Call:.2f}")
print(f"{'Prix Put':<15}{P_Put:.2f}")
print(f"{'Delta Call':<15}{DeltaCall:.4f}")
print(f"{'Delta Put':<15}{DeltaPut:.4f}")

print(f"{'Real Close':<15}{Close:.4f}")
print(f"{'Type option':<15}{Type}")

# Optimisation pour trouver la volatilité implicite
result = minimize(objective_function, vol_initial, bounds=[(0.01, 2)])  # On fixe des bornes raisonnables pour la volatilité

if result.success:
    implied_volatility = result.x[0]
    print(f"{'Volatilité implicite':<15}{implied_volatility:.4f}")
else:
    print("L'optimisation a échoué.")


# Calculer les prix et les deltas avec la volatilité implicite 
P_Call_test, P_Put_test, DeltaCall_test, DeltaPut_test = Pricer(Underlying_close, Strike, r_d, ttm, div_pv, implied_volatility)
print(Underlying_close)
print(Strike)
print(r_d)
print(ttm)
print(div_pv)
print(implied_volatility)
# Affichage des résultats formatés
print(f"{'Paramètre':<15}{'Valeur'}")
print("-" * 25)
print(f"{'Prix Call':<15}{P_Call_test:.2f}")
print(f"{'Prix Put':<15}{P_Put_test:.2f}")
print(f"{'Delta Call':<15}{DeltaCall_test:.4f}")
print(f"{'Delta Put':<15}{DeltaPut_test:.4f}")

print(f"{'Real Close':<15}{Close:.4f}")
print(f"{'Type option':<15}{Type}")

Paramètre      Valeur
-------------------------
Prix Call      7.52
Prix Put       7.98
Delta Call     0.5150
Delta Put      -0.4850
Real Close     0.5100
Type option    CALL
Volatilité implicite0.0462
158.04
158.5
0
0.0603
0
0.046220076861629704
Paramètre      Valeur
-------------------------
Prix Call      0.51
Prix Put       0.97
Delta Call     0.4011
Delta Put      -0.5989
Real Close     0.5100
Type option    CALL


In [107]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv("Data.csv", sep=',')

# Filtrer les lignes où la colonne 'Type' est égale à 'Option'
df_option = df[df['Type'] == 'Option']

# Afficher le DataFrame filtré
df_option.head()


,Code,Instrument_id,Name,Quantity,Close,Currency,Mo,Mf,fx_rate,AUM,...,lower_close,upper_euribor_id,upper_pillar,upper_close,Days_To_Expiration,TypeOption,Close_Associated_option,Type,MV,Weight
118,OPT_EURO-BUND_11/18_CALL_158.5,27085,EURO-BUND 11/18 CALL 158.5,50,0.51,EUR,1.0,1000,1.0,37641000,...,NaN,1637.0,1.0,-0.371,0.0603,CALL,0.065,Option,25500,NaN
121,OPT_SX5E_12/21/2018_PUT_3300.0,27108,SX5E 12/21/2018 PUT 3300.0,150,65.10,EUR,10.0,1,1.0,37641000,...,-0.337,3377.0,3.0,-0.318,0.2137,PUT,129.100,Option,97650,NaN
125,OPT_EURO-BOBL_11/18_CALL_130.75,27172,EURO-BOBL 11/18 CALL 130.75,20,0.17,EUR,1.0,1000,1.0,37641000,...,NaN,1637.0,1.0,-0.371,0.0603,CALL,0.460,Option,3400,NaN
126,OPT_EURO-BUND_11/18_CALL_160.5,27248,EURO-BUND 11/18 CALL 160.5,50,0.09,EUR,1.0,1000,1.0,37641000,...,NaN,1637.0,1.0,-0.371,0.0603,CALL,2.550,Option,4500,NaN
129,OPT_SX5E_11/16/2018_CALL_3400.0,27326,SX5E 11/16/2018 CALL 3400.0,-150,48.70,EUR,10.0,1,1.0,37641000,...,-0.371,7074.0,2.0,-0.337,0.1178,CALL,78.000,Option,-73050,NaN


In [108]:
import numpy as np
import pandas as pd

# Charger le DataFrame
df = pd.read_csv("Data.csv", sep=',')

# Définir les fonctions
def rd(Type, UnderlyingType, month_to_expiration, lower_pillar, lower_close, upper_close):
    if UnderlyingType == 'I':
        time = month_to_expiration - lower_pillar
        rd = lower_close + time * (upper_close - lower_close)
        return rd
    else:
        return 0

def rc(rd_value):
    return np.log(1 + rd_value)

def div_PV(Underlying_Close, UnderlyingType, Strike, Days_To_Expiration, TypeOption, Close, Close_Associated_option):
    if (UnderlyingType == 'FUC'):
        return 0
    else:
        if TypeOption == 'CALL':
            D = Underlying_Close - Strike * np.exp(-Days_To_Expiration) - Close + Close_Associated_option
        elif TypeOption == 'PUT':
            D = Underlying_Close - Strike * np.exp(-Days_To_Expiration) + Close - Close_Associated_option
        else:
            D = 0
        return D

# Appliquer la fonction 'rd' sur chaque ligne
df['rd_value'] = df.apply(lambda row: rd(
    row['Type'], row['UnderlyingType'], row['months_to_expiration'], 
    row['lower_pillar'], row['lower_close'], row['upper_close']
), axis=1)

# Appliquer la fonction 'rc' en utilisant la colonne 'rd_value'
df['rc_value'] = df['rd_value'].apply(rc)

# Appliquer la fonction 'div_PV' sur chaque ligne
df['div_PV_value'] = df.apply(lambda row: div_PV(
    row['Underlying_Close'],row['UnderlyingType'], row['Strike'], row['Days_To_Expiration'], 
    row['TypeOption'], row['Close'], row['Close_Associated_option']
), axis=1)

# Filtrer les lignes où la colonne 'Type' est égale à 'Option'
df_option = df[df['Type'] == 'Option']

# Afficher le DataFrame filtré
df_option.head()

,Code,Instrument_id,Name,Quantity,Close,Currency,Mo,Mf,fx_rate,AUM,...,upper_close,Days_To_Expiration,TypeOption,Close_Associated_option,Type,MV,Weight,rd_value,rc_value,div_PV_value
118,OPT_EURO-BUND_11/18_CALL_158.5,27085,EURO-BUND 11/18 CALL 158.5,50,0.51,EUR,1.0,1000,1.0,37641000,...,-0.371,0.0603,CALL,0.065,Option,25500,NaN,0.000000,0.000000,0.000000
121,OPT_SX5E_12/21/2018_PUT_3300.0,27108,SX5E 12/21/2018 PUT 3300.0,150,65.10,EUR,10.0,1,1.0,37641000,...,-0.318,0.2137,PUT,129.100,Option,97650,NaN,-0.325600,-0.393932,646.030935
125,OPT_EURO-BOBL_11/18_CALL_130.75,27172,EURO-BOBL 11/18 CALL 130.75,20,0.17,EUR,1.0,1000,1.0,37641000,...,-0.371,0.0603,CALL,0.460,Option,3400,NaN,0.000000,0.000000,0.000000
126,OPT_EURO-BUND_11/18_CALL_160.5,27248,EURO-BUND 11/18 CALL 160.5,50,0.09,EUR,1.0,1000,1.0,37641000,...,-0.371,0.0603,CALL,2.550,Option,4500,NaN,0.000000,0.000000,0.000000
129,OPT_SX5E_11/16/2018_CALL_3400.0,27326,SX5E 11/16/2018 CALL 3400.0,-150,48.70,EUR,10.0,1,1.0,37641000,...,-0.337,0.1178,CALL,78.000,Option,-73050,NaN,-0.356268,-0.440472,382.209047


In [109]:
# Fonction pour calculer le taux de dividende
def div_yield(div_pv, S, ttm):
    return div_pv / (S * ttm)

# Fonction de pricing d'option Black-Scholes
def Pricer(S, K, r_d, ttm, div_pv, vol):
    div_yield_value = div_yield(div_pv, S, ttm)
    r_c = np.log(1 + r_d)
    d1 = (np.log(S / K) + (r_c - div_yield_value + 0.5 * vol ** 2) * ttm) / (vol * np.sqrt(ttm))
    d2 = d1 - vol * np.sqrt(ttm)
    P_Call = S * np.exp(-div_yield_value * ttm) * norm.cdf(d1) - np.exp(-r_c * ttm) * K * norm.cdf(d2)
    P_Put = -S * np.exp(-div_yield_value * ttm) * norm.cdf(-d1) + np.exp(-r_c * ttm) * K * norm.cdf(-d2)
    DeltaCall = np.exp(-div_yield_value * ttm) * norm.cdf(d1)
    DeltaPut = -np.exp(-div_yield_value * ttm) * norm.cdf(-d1)
    return P_Call, P_Put, DeltaCall, DeltaPut

# Fonction pour calculer la différence entre le prix réel et le prix de l'option calculé
def objective_function(vol, Underlying_close, Strike, r_d, ttm, div_pv, Close, TypeOption):
    P_Call, P_Put, _, _ = Pricer(Underlying_close, Strike, r_d, ttm, div_pv, vol)
    if TypeOption == 'CALL':
        return (P_Call - Close) ** 2
    elif TypeOption == 'PUT':
        return (P_Put - Close) ** 2
    else :
        return 0

# Fonction pour calculer la volatilité implicite pour une ligne
def calculate_volatility_for_row(row):
    # Récupérer les paramètres de la ligne
    Close = row['Close']
    Underlying_close = row['Underlying_Close']
    Strike = row['Strike']
    ttm = row['Days_To_Expiration'] 
    r_d = row['rd_value']
    div_pv = row['div_PV_value']
    TypeOption = row['TypeOption']
    vol_initial = 0.30  # Valeur initiale pour l'optimisation
    
    # Optimiser pour trouver la volatilité implicite
    result = minimize(objective_function, vol_initial, args=(Underlying_close, Strike, r_d, ttm, div_pv, Close, TypeOption), bounds=[(0.01, 2)])
    implied_vol = result.x[0] if result.success else np.nan
    
    # Calculer le prix de l'option et les deltas avec cette volatilité
    P_Call, P_Put, DeltaCall, DeltaPut = Pricer(Underlying_close, Strike, r_d, ttm, div_pv, implied_vol)
    
    # Retourner les valeurs pour cette ligne
    return pd.Series({'implied_volatility': implied_vol, 'P_Call': P_Call, 'P_Put': P_Put, 'DeltaCall': DeltaCall, 'DeltaPut': DeltaPut})

# Appliquer la fonction à chaque ligne et ajouter les résultats dans de nouvelles colonnes
df[['implied_volatility', 'P_Call', 'P_Put', 'DeltaCall', 'DeltaPut']] = df.apply(calculate_volatility_for_row, axis=1)




# Filtrer les lignes où la colonne 'Type' est égale à 'Option'
df_option = df[df['Type'] == 'Option']

# Afficher le DataFrame filtré
df_option.head()


,Code,Instrument_id,Name,Quantity,Close,Currency,Mo,Mf,fx_rate,AUM,...,MV,Weight,rd_value,rc_value,div_PV_value,implied_volatility,P_Call,P_Put,DeltaCall,DeltaPut
118,OPT_EURO-BUND_11/18_CALL_158.5,27085,EURO-BUND 11/18 CALL 158.5,50,0.51,EUR,1.0,1000,1.0,37641000,...,25500,NaN,0.000000,0.000000,0.000000,0.046220,5.099999e-01,0.970000,4.011383e-01,-0.598862
121,OPT_SX5E_12/21/2018_PUT_3300.0,27108,SX5E 12/21/2018 PUT 3300.0,150,65.10,EUR,10.0,1,1.0,37641000,...,97650,NaN,-0.325600,-0.393932,646.030935,0.010000,0.000000e+00,802.718147,0.000000e+00,-0.825792
125,OPT_EURO-BOBL_11/18_CALL_130.75,27172,EURO-BOBL 11/18 CALL 130.75,20,0.17,EUR,1.0,1000,1.0,37641000,...,3400,NaN,0.000000,0.000000,0.000000,0.022854,1.699999e-01,0.460000,3.472141e-01,-0.652786
126,OPT_EURO-BUND_11/18_CALL_160.5,27248,EURO-BUND 11/18 CALL 160.5,50,0.09,EUR,1.0,1000,1.0,37641000,...,4500,NaN,0.000000,0.000000,0.000000,0.010000,9.423668e-12,2.460000,1.599864e-10,-1.000000
129,OPT_SX5E_11/16/2018_CALL_3400.0,27326,SX5E 11/16/2018 CALL 3400.0,-150,48.70,EUR,10.0,1,1.0,37641000,...,-73050,NaN,-0.356268,-0.440472,382.209047,0.508413,4.870000e+01,616.057031,1.640593e-01,-0.728873


In [110]:
# Définir la fonction Weight_option
def Weight_option(quantity, Underlying_Close, Mf, Mo, fx,DeltaCall, DeltaPut, TypeOption, AUM):
    if (TypeOption == 'CALL'):
        return (quantity*Mo*Mf*fx*DeltaCall*Underlying_Close)/AUM
    elif (TypeOption == 'PUT'):
        return (quantity*Mo*Mf*fx*DeltaPut*Underlying_Close)/AUM

# S'assurer que la colonne 'Weight' existe et initialiser à `None` si elle n'existe pas encore
if 'Weight' not in df.columns:
    df['Weight'] = None

# Appliquer la fonction uniquement aux lignes où 'Type' est égal à 'Option' sans modifier les autres lignes
df['Weight'] = df.apply(
    lambda row: Weight_option(row['Quantity'], row['Underlying_Close'], row['Mf'], row['Mo'], row['fx_rate'],row['DeltaCall'], row['DeltaPut'],row['TypeOption'], row['AUM'])
    if row['Type'] == 'Option' else row['Weight'],  # Conserver les valeurs actuelles pour les autres lignes
    axis=1
)

# Filtrer les lignes où la colonne 'Type' est égale à 'Option'
df_option = df[df['Type'] == 'Option']

# Afficher le DataFrame filtré
df_option.head()


,Code,Instrument_id,Name,Quantity,Close,Currency,Mo,Mf,fx_rate,AUM,...,MV,Weight,rd_value,rc_value,div_PV_value,implied_volatility,P_Call,P_Put,DeltaCall,DeltaPut
118,OPT_EURO-BUND_11/18_CALL_158.5,27085,EURO-BUND 11/18 CALL 158.5,50,0.51,EUR,1.0,1000,1.0,37641000,...,25500,8.421123e-02,0.000000,0.000000,0.000000,0.046220,5.099999e-01,0.970000,4.011383e-01,-0.598862
121,OPT_SX5E_12/21/2018_PUT_3300.0,27108,SX5E 12/21/2018 PUT 3300.0,150,65.10,EUR,10.0,1,1.0,37641000,...,97650,-1.110670e-01,-0.325600,-0.393932,646.030935,0.010000,0.000000e+00,802.718147,0.000000e+00,-0.825792
125,OPT_EURO-BOBL_11/18_CALL_130.75,27172,EURO-BOBL 11/18 CALL 130.75,20,0.17,EUR,1.0,1000,1.0,37641000,...,3400,2.406820e-02,0.000000,0.000000,0.000000,0.022854,1.699999e-01,0.460000,3.472141e-01,-0.652786
126,OPT_EURO-BUND_11/18_CALL_160.5,27248,EURO-BUND 11/18 CALL 160.5,50,0.09,EUR,1.0,1000,1.0,37641000,...,4500,3.358606e-11,0.000000,0.000000,0.000000,0.010000,9.423668e-12,2.460000,1.599864e-10,-1.000000
129,OPT_SX5E_11/16/2018_CALL_3400.0,27326,SX5E 11/16/2018 CALL 3400.0,-150,48.70,EUR,10.0,1,1.0,37641000,...,-73050,-2.206557e-02,-0.356268,-0.440472,382.209047,0.508413,4.870000e+01,616.057031,1.640593e-01,-0.728873


In [113]:
# Liste des colonnes que vous souhaitez inclure dans le DataFrame final
colonnes_finales = [
    'Code',
    'Name',
    'Quantity',
    'Close',
    'Currency',
    'fx_rate',
    'MV',
    'Weight'
]

# Créer le DataFrame final en ne sélectionnant que les colonnes désirées
df_final = df[colonnes_finales]

# Afficher les premières lignes du DataFrame final
print(df_final.head())

# Sauvegarder le DataFrame final au format CSV
df_final.to_csv('dataset_final.csv', index=False)

# Filtrer les lignes où la colonne 'Type' est égale à 'Option'
df_option = df[df['Type'] == 'Option']

# Afficher le DataFrame filtré
df_option.head()



           Code                         Name  Quantity   Close Currency  \
0  FR0000045072           CREDIT AGRICOLE SA     14000   12.67      EUR   
1  FR0010340141                          ADP      1400  189.10      EUR   
2  FR0000120073                  AIR LIQUIDE      3300  113.10      EUR   
3  DE0008404005               ALLIANZ SE-REG      2400  194.36      EUR   
4  DE0005190003  BAYERISCHE MOTOREN WERKE AG      2000   78.25      EUR   

   fx_rate      MV    Weight  
0      1.0  177380  0.004712  
1      1.0  264740  0.007033  
2      1.0  373230  0.009916  
3      1.0  466464  0.012392  
4      1.0  156500  0.004158  


,Code,Instrument_id,Name,Quantity,Close,Currency,Mo,Mf,fx_rate,AUM,...,MV,Weight,rd_value,rc_value,div_PV_value,implied_volatility,P_Call,P_Put,DeltaCall,DeltaPut
118,OPT_EURO-BUND_11/18_CALL_158.5,27085,EURO-BUND 11/18 CALL 158.5,50,0.51,EUR,1.0,1000,1.0,37641000,...,25500,8.421123e-02,0.000000,0.000000,0.000000,0.046220,5.099999e-01,0.970000,4.011383e-01,-0.598862
121,OPT_SX5E_12/21/2018_PUT_3300.0,27108,SX5E 12/21/2018 PUT 3300.0,150,65.10,EUR,10.0,1,1.0,37641000,...,97650,-1.110670e-01,-0.325600,-0.393932,646.030935,0.010000,0.000000e+00,802.718147,0.000000e+00,-0.825792
125,OPT_EURO-BOBL_11/18_CALL_130.75,27172,EURO-BOBL 11/18 CALL 130.75,20,0.17,EUR,1.0,1000,1.0,37641000,...,3400,2.406820e-02,0.000000,0.000000,0.000000,0.022854,1.699999e-01,0.460000,3.472141e-01,-0.652786
126,OPT_EURO-BUND_11/18_CALL_160.5,27248,EURO-BUND 11/18 CALL 160.5,50,0.09,EUR,1.0,1000,1.0,37641000,...,4500,3.358606e-11,0.000000,0.000000,0.000000,0.010000,9.423668e-12,2.460000,1.599864e-10,-1.000000
129,OPT_SX5E_11/16/2018_CALL_3400.0,27326,SX5E 11/16/2018 CALL 3400.0,-150,48.70,EUR,10.0,1,1.0,37641000,...,-73050,-2.206557e-02,-0.356268,-0.440472,382.209047,0.508413,4.870000e+01,616.057031,1.640593e-01,-0.728873
